In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from nppm_latam_optuner.optuner import Optuner

/home/ec2-user/SageMaker/totalloss_HDI/totallosshdi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pwd

/home/ec2-user/SageMaker/totalloss_HDI


In [3]:
!cd ..

In [4]:
!aws s3 cp nppm_latam_optuner/nppm_latam_optuner/optuner.py s3://hdi-sagemaker-project-co/arquitectura/scripts/optuner.py


The user-provided path nppm_latam_optuner/nppm_latam_optuner/optuner.py does not exist.


In [5]:
df = pd.read_excel(
    'perdidas_daños_HDI.xlsx',
   # index_col=0
)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12960 entries, 0 to 12959
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Amparo                   12960 non-null  object 
 1   placa                    12960 non-null  object 
 2   siniestro                12960 non-null  int64  
 3   poliza                   12960 non-null  int64  
 4   estado_siniestro         12960 non-null  object 
 5   causa_siniestro          12960 non-null  object 
 6   cod_marca                12960 non-null  int64  
 7   cod_clase                12960 non-null  int64  
 8   cod_tipo                 12960 non-null  int64  
 9   modelo                   12960 non-null  int64  
 10  valor_asegurado_sin_acc  12960 non-null  float64
 11  valor_veh_fasecolda      12960 non-null  int64  
 12  valor_accesorios         12960 non-null  float64
 13  color                    12960 non-null  object 
 14  perdida_total         

#### Pre

In [6]:
df['descripcion']=df['descripcion'].str.lower()
df['descripcion']=df['descripcion'].str.replace("(?i)contact center.*?;", "", regex=True)
df['descripcion']=df['descripcion'].str.replace("(?i)contac center.*?;", "", regex=True)
df['descripcion']=df['descripcion'].str.replace("(?i)contactcenter.*?;", "", regex=True)
df['descripcion']=df['descripcion'].str.replace("(?i)contaccenter.*?;", "", regex=True)

df['descripcion']=df['descripcion'].str.replace("(?i)contact.*?;", "", regex=True)
df['descripcion']=df['descripcion'].str.replace("(?i)contac.*?;", "", regex=True)



df['descripcion']=df['descripcion'].str.replace("(?i)call center.*?;", "", regex=True)
df['descripcion']=df['descripcion'].str.replace("(?i)callcenter.*?;", "", regex=True)
df['descripcion']=df['descripcion'].str.replace("(?i)cal center.*?;", "", regex=True)
df['descripcion']=df['descripcion'].str.replace("(?i)call centr.*?;", "", regex=True)


df['descripcion']=df['descripcion'].str.replace("(?i)cente.*?;", "", regex=True)


In [7]:
df['descripcion']

0        ecduartec  nos dirigamos hacia bogot en la rut...
1        cclopezo  cuando iba por la carretera habia un...
2        nbarrazam  iba a girar a la izquierda y una mo...
3        ecduartec  eran las 12 de la noche ya a esa ho...
4        jsquinemev   el asegurado llevaba un tercero a...
                               ...                        
12955     ibamos hacia el centro de bogota por la cra 1...
12956     yo iba bajando por el puente de la 127 llegan...
12957     asegurado indica que iba hacia la avenida la ...
12958     asegurada indica  que iba por la via marinill...
12959     me dirigia hacia marinilla y una buseta urban...
Name: descripcion, Length: 12960, dtype: object

In [8]:
df["Codigo"]=df["cod_marca"].astype("str").str.zfill(3)+df["cod_clase"].astype("str").str.zfill(5)
df["Codigo"]

0        09201266
1        01601291
2        01601341
3        01601325
4        01601345
           ...   
12955    06401233
12956    00808041
12957    08001214
12958    01601339
12959    01601339
Name: Codigo, Length: 12960, dtype: object

In [9]:
fase=pd.read_csv("Guia_CSV_333.csv")
fase["Codigo"]=fase["Codigo"].astype("str").str.zfill(8)

In [10]:
fase["Codigo"]

0        00101001
1        00208003
2        00208004
3        00206001
4        00301001
           ...   
16873    46717001
16874    46817001
16875    46817002
16876    46925001
16877    47022001
Name: Codigo, Length: 16878, dtype: object

In [11]:
fase_2 = fase[['Codigo', 'Peso', 'Transmision', 'Cilindraje', 'Nacionalidad','CapacidadPasajeros', 'CapacidadCarga']]
df = pd.merge(df, fase_2, on='Codigo', how='left')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12960 entries, 0 to 12959
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Amparo                   12960 non-null  object 
 1   placa                    12960 non-null  object 
 2   siniestro                12960 non-null  int64  
 3   poliza                   12960 non-null  int64  
 4   estado_siniestro         12960 non-null  object 
 5   causa_siniestro          12960 non-null  object 
 6   cod_marca                12960 non-null  int64  
 7   cod_clase                12960 non-null  int64  
 8   cod_tipo                 12960 non-null  int64  
 9   modelo                   12960 non-null  int64  
 10  valor_asegurado_sin_acc  12960 non-null  float64
 11  valor_veh_fasecolda      12960 non-null  int64  
 12  valor_accesorios         12960 non-null  float64
 13  color                    12960 non-null  object 
 14  perdida_total         

#### Modelo

In [13]:
X = df.drop(columns=['perdida_total','placa','siniestro','poliza','id','Amparo','estado_siniestro','color','Codigo'])

y = df.perdida_total
y.value_counts()

perdida_total
0    11650
1     1310
Name: count, dtype: int64

In [14]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12960 entries, 0 to 12959
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   causa_siniestro          12960 non-null  object 
 1   cod_marca                12960 non-null  int64  
 2   cod_clase                12960 non-null  int64  
 3   cod_tipo                 12960 non-null  int64  
 4   modelo                   12960 non-null  int64  
 5   valor_asegurado_sin_acc  12960 non-null  float64
 6   valor_veh_fasecolda      12960 non-null  int64  
 7   valor_accesorios         12960 non-null  float64
 8   descripcion              12959 non-null  object 
 9   Peso                     12958 non-null  float64
 10  Transmision              12907 non-null  object 
 11  Cilindraje               12958 non-null  float64
 12  Nacionalidad             12958 non-null  object 
 13  CapacidadPasajeros       12958 non-null  float64
 14  CapacidadCarga        

In [15]:
# !aws s3 cp s3://hdi-sagemaker-project-co/arquitectura/modulos-spacy/stopwords.zip stopwords.zip

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42)

skf = StratifiedKFold(n_splits=4, random_state=0, shuffle=True)

opt = Optuner(study_name='co_toralloss_6', prediction_task='clf',
              X=X_train, y=y_train,
              num_cols=['valor_asegurado_sin_acc', 'valor_veh_fasecolda','valor_accesorios','Peso','modelo','Cilindraje','CapacidadPasajeros','CapacidadCarga'],
              cat_cols=['causa_siniestro','cod_marca','cod_tipo','cod_clase','Transmision','Nacionalidad'],
              text_cols=['descripcion'],
              language='spa',
              cv=skf, scoring='average_precision', direction='maximize',
              model_n_jobs=9, cross_val_n_jobs=4)

opt.optune(100)

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     self signed certificate in certificate chain
[nltk_data]     (_ssl.c:997)>


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/home/ec2-user/nltk_data'
    - '/home/ec2-user/SageMaker/totalloss_HDI/totallosshdi/nltk_data'
    - '/home/ec2-user/SageMaker/totalloss_HDI/totallosshdi/share/nltk_data'
    - '/home/ec2-user/SageMaker/totalloss_HDI/totallosshdi/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
import joblib

optuned_model = joblib.load('co_toralloss_6.joblib')
type(optuned_model)

In [ ]:
from sklearn.metrics import average_precision_score

probas = optuned_model.predict_proba(X_test)

X_test.shape, probas.shape

In [ ]:
average_precision_score(y_test, probas[:, 1])

In [ ]:
X_test['Probabilidades'] = probas[:, 1]  

In [ ]:
X_test = X_test.sort_values( by = ['Probabilidades'], ascending = False).to_excel('prueba_fija.xlsx', index = False)